In [ ]:
import os

import pandas as pd

In [ ]:
SAVE_OUTPUT = False

FN_RL_CSV = "returns_y30_best_model.bson.csv"

PATH_MEV_MODELS = os.path.join("..", "data", "models", "mev_nn")

PATH_FILE = os.path.join(PATH_MEV_MODELS, "2024-01-03T12-42-32", FN_RL_CSV)
PATH_OUTPUT = "tmp"

In [ ]:
df = pd.read_csv(PATH_FILE)

df["target"] /= 10  # convert mm to cm

In [ ]:
df

In [ ]:
if SAVE_OUTPUT:
    os.makedirs(PATH_OUTPUT)

# Plot Return levels high resolution

In [ ]:
import matplotlib.pyplot as plt

from mpl_toolkits.basemap import Basemap

In [ ]:
fig = plt.figure(figsize = (15, 10))

#initialize the Basemap
m = Basemap(projection = 'lcc', resolution='f', lat_0=47.5, lon_0=13.3, width=0.6E6, height=3.7E5)
m.drawmapboundary()
m.drawcountries(linewidth=2)

m.scatter(df['lon'], df['lat'], c=df["target"], cmap="jet", s=0.5, latlon=True, vmin=1, vmax=6)

plt.colorbar(label='MEHS', extend="max")

if SAVE_OUTPUT:
    plt.savefig(os.path.join(PATH_OUTPUT, f"hailriskat_rl.png"), bbox_inches="tight")
    plt.close()
else:
    plt.show()

# HORA Classification

In [ ]:
import numpy as np

In [ ]:
df.loc[:, 'cat_string'] = np.select(
    [
        (df["target"] > 0) & (df["target"] < 2), 
        (df["target"] >= 2) & (df["target"] < 3), 
        (df["target"] >= 3) & (df["target"] < 4), 
        (df["target"] >= 4) & (df["target"] < 5), 
        (df["target"] >= 5),
    ], 
    [
        'HW 2',
        'HW 3', 
        'HW 4',
        'HW 5',
        'HW 6',
    ], 
    default='ERROR'
)

# Plot Return levels HORA classification

In [ ]:
colors = {
        'HW 2' : 'black',
        'HW 3' : 'dodgerblue', 
        'HW 4' : 'springgreen',
        'HW 5' : 'orange',
        'HW 6' : 'darkred',
}

ax = plt.figure(figsize = (15, 10))

#initialize the Basemap
m = Basemap(projection = 'lcc', resolution='f', lat_0=47.5, lon_0=13.3, width=0.6E6, height=3.7E5)
m.drawmapboundary()
m.drawcountries(linewidth=2)

for cat in np.unique(df['cat_string']):
    return_data_cat = df.query(f'cat_string == "{cat}"') 
    
    m.scatter(return_data_cat['lon'], return_data_cat['lat'], c=colors[cat], latlon=True, label=cat, marker=',', s=0.7, alpha=0.7)

# Plot legend.
ax.legend(loc='center left', bbox_to_anchor=(0.9, 0.5), markerscale=10)

if SAVE_OUTPUT:
    plt.savefig(os.path.join(PATH_OUTPUT, f"hailriskat_rl_hora_classification.png"), bbox_inches="tight")
    plt.close()
else:
    plt.show()

# Hail days

In [ ]:
df_haildays = pd.read_csv("haildays_2009-2022_orig_data.csv")

In [ ]:
df_haildays

In [ ]:
fig = plt.figure(figsize = (15, 10))

m = Basemap(projection = 'lcc', resolution='f', lat_0 = 47.5, lon_0 = 13.3, width=0.6E6, height=3.7E5)
m.drawmapboundary()
m.drawcountries(linewidth=2)

m.scatter(df_haildays['lon'], df_haildays['lat'], c=df_haildays['hail_events'], cmap="nipy_spectral", s=2, latlon=True, vmin=1)
plt.colorbar(label="Hail days", extend="max")
plt.title("Hail days")

if SAVE_OUTPUT:
    plt.savefig(os.path.join(PATH_OUTPUT, "haildays_2009-2022_orig_data.png"), bbox_inches="tight")
    plt.close()
else:
    plt.show()

# Compare HORA classification of two models

In [ ]:
PATH_FILE_COMPARE = os.path.join("..", "data", "models", "mev_nn", "2024-01-23T11-07-00", FN_RL_CSV)
df_compare = pd.read_csv(PATH_FILE_COMPARE)
df_compare["target"] /= 10  # convert mm to cm

In [ ]:
def df_enrich_by_cat(df):
    tmpdf = df.copy()
    
    tmpdf.loc[:, 'cat'] = np.select(
        [
            (tmpdf["target"] > 0) & (tmpdf["target"] < 2), 
            (tmpdf["target"] >= 2) & (tmpdf["target"] < 3), 
            (tmpdf["target"] >= 3) & (tmpdf["target"] < 4), 
            (tmpdf["target"] >= 4) & (tmpdf["target"] < 5), 
            (tmpdf["target"] >= 5),
        ], 
        [
            0,
            1, 
            2,
            3,
            4,
        ], 
        default=-1
    )

    return tmpdf

In [ ]:
df1 = df_enrich_by_cat(df_compare)
df2 = df_enrich_by_cat(df_compare)

df1.loc[:, "diff_class"] = df1["cat"] - df2["cat"]

In [ ]:
colors = {
        -3 : 'darkred',
        -2 : 'orangered', 
        -1 : 'gold',
         0 : 'limegreen',
         1 : 'cornflowerblue',
         2 : 'blue',
         3 : 'magenta',
         4 : 'darkviolet',
         5 : 'darkmagenta',
}

ax = plt.figure(figsize = (15, 10))

#initialize the Basemap
m = Basemap(projection = 'lcc', resolution='f', lat_0=47.5, lon_0=13.3, width=0.6E6, height=3.7E5)
m.drawmapboundary()
m.drawcountries(linewidth=2)

for cat in np.unique(df1['diff_class']):
    return_data_cat = df1.query(f'diff_class == {cat}') 
    
    m.scatter(return_data_cat['lon'], return_data_cat['lat'], c=colors[cat], latlon=True, label=cat, marker=',', s=0.7, alpha=0.7)

# Plot legend.
ax.legend(loc='center left', bbox_to_anchor=(0.9, 0.5), markerscale=10)

if SAVE_OUTPUT:
    plt.savefig(os.path.join(PATH_OUTPUT, f"hailriskat_rl_hora_classification_model_compare.png"), bbox_inches="tight")
    plt.close()
else:
    plt.show()